# SQL in Pandas with SQLAlchemy

We will use the "sshtunnel" library to connect to our remote AWS instance and then pull some data into Pandas using SQLAlchemy and Pyscopg2.

## Creating the SSH Tunnel

The "sshtunnel" library can read an SSH config file, so creating a tunnel is quite easy assuming SSH keys are setup and the SSH config entry has been created. With this setup, the "sshtunnel" library automatically determines what the address of the local port should be.

If you are having problems with the password on SSH , in command line type:

    '''ssh-keygen -p'''
    
- First prompt is directory of key, if the guess is correct then hit enter
- Enter your old passpharse
- Enter nothing for the new passphrase
- You should then be able to tunnel / connect to AWS without a passphrase prompt

https://stackoverflow.com/questions/112396/how-do-i-remove-the-passphrase-for-the-ssh-key-without-having-to-create-a-new-ke

In [1]:
from sshtunnel import SSHTunnelForwarder

AWS_IP_ADDRESS = '34.215.20.24'
AWS_USERNAME = 'roberto'
SSH_KEY_PATH = '/Users/robertomac/.ssh/id_rsa'

server = SSHTunnelForwarder(
    AWS_IP_ADDRESS,
    ssh_username=AWS_USERNAME,
    ssh_pkey=SSH_KEY_PATH,
    remote_bind_address=('localhost', 5432),
)

server.start()
print(server.is_active, server.is_alive, server.local_bind_port)

True True 51377


##  Connecting via Python
We'll be using a Psycopg2 connector alongside SQLAlchemy to connect to this database.

* **SQLAlchemy:** generates SQL statements
* **Psycopg2:** sends the SQL statements to the Postgres database

    Let's make the connection to the database. Note that the IP address of the Postgres database is 'localhost' and the port is set to whatever the `server` connection above contains. This is because we have used the SSH tunnel to create a connection between the AWS instance and our computer. SSH tunnels enable remote instances to behave as if they are *local*.

In [2]:
from sqlalchemy import create_engine

# Postgres username, password, and database name
POSTGRES_IP_ADDRESS = 'localhost' ## This is localhost because SSH tunnel is active
POSTGRES_PORT = str(server.local_bind_port)
POSTGRES_USERNAME = 'roberto'     ## CHANGE THIS TO YOUR POSTGRES USERNAME
POSTGRES_PASSWORD = 'roberto' ## CHANGE THIS TO YOUR POSTGRES PASSWORD
POSTGRES_DBNAME = 'baseball'

# A long string that contains the necessary Postgres login information
postgres_str = ('postgresql://{username}:{password}@{ipaddress}:{port}/{dbname}'
                .format(username=POSTGRES_USERNAME, 
                        password=POSTGRES_PASSWORD,
                        ipaddress=POSTGRES_IP_ADDRESS,
                        port=POSTGRES_PORT,
                        dbname=POSTGRES_DBNAME))

# Create the connection
cnx = create_engine(postgres_str)

## Load Some Data!

Pandas has a `read_sql_query` method that will pass a SQL statement to a database connection. Here is an example from the all-star table.

In [3]:
import pandas as pd

pd.read_sql_query('''SELECT * FROM allstarfull LIMIT 5;''', cnx)

,playerid,yearid,gamenum,gameid,teamid,lgid,gp,startingpos
0,gomezle01,1933,0,ALS193307060,NYA,AL,1,1
1,ferreri01,1933,0,ALS193307060,BOS,AL,1,2
2,gehrilo01,1933,0,ALS193307060,NYA,AL,1,3
3,gehrich01,1933,0,ALS193307060,DET,AL,1,4
4,dykesji01,1933,0,ALS193307060,CHA,AL,1,5


And another from the schools table.

In [4]:
pd.read_sql_query('''SELECT * FROM schools LIMIT 5;''', cnx)

,schoolid,schoolname,schoolcity,schoolstate,schoolnick
0,abilchrist,Abilene Christian University,Abilene,TX,Wildcats
1,adelphi,Adelphi University,Garden City,NY,Panthers
2,adrianmi,Adrian College,Adrian,MI,Bulldogs
3,airforce,United States Air Force Academy,Colorado Springs,CO,Falcons
4,akron,University of Akron,Akron,OH,Zips


More sophisticated queries can also be used. This example finds the states with the most schools.

In [5]:
sql_query = '''SELECT schoolstate as state, Count(schoolid) as ct 
               FROM schools 
               GROUP BY state 
               ORDER BY ct DESC 
               LIMIT 5;'''

pd.read_sql_query(sql_query, cnx)

,state,ct
0,PA,57
1,CA,48
2,NY,45
3,TX,41
4,OH,33


Finally, this example finds five players from the year 1985 whose salary was above $500,000.

In [6]:
sql_query = '''SELECT playerid, salary 
               FROM Salaries 
               WHERE yearid = '1985' AND salary > '500000'
               ORDER BY salary DESC
               LIMIT 5;'''

pd.read_sql_query(sql_query, cnx)

,playerid,salary
0,schmimi01,2130300.0
1,cartega01,2028571.0
2,fostege01,1942857.0
3,winfida01,1795704.0
4,gossari01,1713333.0


## Close Server Connection

Finally, we should close the server connection when complete.

In [7]:
server.close()